In [1]:
# Params
input_size = (100,100)
batch_size = 512
num_workers = 10
learning_rate = 1e-03
SGD_momentum = 0.9
epochs = 10

In [2]:
import torch

print("torch-version:", torch.__version__)
print("Available GPU:", torch.cuda.current_device())

torch-version: 1.1.0
Available GPU: 0


## CIFAR 10

In [3]:
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.Resize(input_size), # Must be ahead of transforms.ToTensor()
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print()
print(trainset.__getitem__(0)[0].size(), trainset.__len__())
print(testset.__getitem__(0)[0].size(), testset.__len__())

Files already downloaded and verified
Files already downloaded and verified

torch.Size([3, 100, 100]) 50000
torch.Size([3, 100, 100]) 10000


## Model

In [4]:
from googlenet_pytorch import *
    
net = googlenet(pretrained=False, progress=True, num_classes=10, aux_logits=True, transform_input=False, init_weights=True).cuda()

#for i in net.named_children():
#    print(i)

In [5]:
from torch.optim.lr_scheduler import StepLR

#criterion = torch.nn.BCEWithLogitsLoss()
criterion = torch.nn.CrossEntropyLoss()
#criterion = torch.nn.BCELoss(reduction='sum')
#optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=SGD_momentum)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=8, gamma=0.96)

## Train

In [6]:
times = 1
aux_loss_weight = 0.3

for epoch in range(epochs):  # loop over the dataset multiple times
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs, aux_2, aux_1 = net(inputs) #tuple: (input, aux2, aux1)
        
        loss1 = criterion(outputs, labels)
        loss2 = criterion(aux_2, labels)
        loss3 = criterion(aux_1, labels)
        
        loss = loss1 + aux_loss_weight*loss2 + aux_loss_weight*loss3
        loss.backward()
        ##scheduler.step() #학습이 안됨.
        optimizer.step()
        
        running_loss += loss.item()
        
    if epoch % times == 0:
        print('[%d] loss: %.3f' %(epoch, running_loss / times))
        running_loss = 0.0
        #torch.save(model,'./data/cifar_model.pkl')

print('\nFinished Training')

[0] loss: 242.538
[1] loss: 168.193
[2] loss: 130.427
[3] loss: 107.401
[4] loss: 89.477
[5] loss: 74.862
[6] loss: 62.852
[7] loss: 53.657
[8] loss: 46.649
[9] loss: 38.669

Finished Training


## Test

In [7]:
correct = 0
total = 0
net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda() #.cpu()
        labels = labels.cuda() #.cpu()
        
        outputs = net(images)
        
        _, predicted = torch.max(outputs, dim=1)
        #_, predicted = outputs.max(1)
        
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' %(100 * correct / total))

Accuracy of the network on the 10000 test images: 79 %
